Let's first see if we are using GPU or not. To get all the information about GPU, use the following code.

In [48]:
!nvidia-smi

Sun Apr  9 12:08:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |   4793MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Now let's install the setfit library created by Hugging Face.

In [4]:
!pip install setfit datasets sentence_transformers

In [18]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from sklearn.metrics import precision_score, recall_score, f1_score

In [8]:
from google.colab import files
uploades = files.upload()

Saving yelp_labelled.csv to yelp_labelled.csv


Let's read the data.

In [2]:
df = load_dataset("SetFit/SentEval-CR")

  0%|          | 0/2 [00:00<?, ?it/s]

The dataset should be in the following format to run the huggingface library on it. 
If your dataset is not in this format then you can convert your data using same load_dataset()

In [3]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 3012
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 753
    })
})

Now to check how good the SetFit is...let's only have 8 samples from our dataset. I am using 8 because 8 was recommanded in the few-shot learning paper.

In [ ]:
df['train'].shuffle(seed= 11).select(range(16*2))['label']

In [57]:
training = df['train'].shuffle(seed= 11).select(range(8*2))
testing = df['test']

In [58]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [59]:
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=training,
    eval_dataset=testing,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

In [60]:
# Train and evaluate!
%%time
trainer.train()

***** Running training *****
  Num examples = 10240
  Num epochs = 1
  Total optimization steps = 640
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/640 [00:00<?, ?it/s]

CPU times: user 3min 5s, sys: 11.9 s, total: 3min 17s
Wall time: 3min 21s


In [61]:
%time
metrics = trainer.evaluate()

***** Running evaluation *****


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.2 µs


In [62]:
metrics

{'accuracy': 0.9216467463479415}

In [63]:
trainer.evaluate()

***** Running evaluation *****


{'accuracy': 0.9216467463479415}

In [64]:
preds = model.predict(testing['text'])

In [65]:
print('Precision score:', precision_score(testing['label'], preds))
print('Recall score:', recall_score(testing['label'], preds))
print('F1 score:', f1_score(testing['label'], preds))

Precision score: 0.9214876033057852
Recall score: 0.9550321199143469
F1 score: 0.9379600420609885
